NameError: name 'python' is not defined

In [2]:
import pandas as pd

# Get training dataset
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
df_train.head()

# Get testing dataset
df_test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])
df_test.head()

/Users/huy.suy05./Documents/Projects/LLM_Evals_On_Imbalanced_Datatset/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,text,label
0,Fears for T N pension after talks Unions repre...,2
1,The Race is On: Second Private Team Sets Launc...,3
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3


In [41]:
from dotenv import load_dotenv
import os
import requests
import time
import json


load_dotenv()
API_KEY = os.getenv("API_KEY")

# Sample out an balanced training data
n_rows_per_class = 1000
balanced_dfs = []

for label in sorted(df_train["label"].unique()):
    class_samples = df_train[df_train["label"] == label]
    balanced_dfs.append(class_samples.sample(n_rows_per_class, random_state=42))

balanced_data = pd.concat(balanced_dfs)

# Sample out an imbalanced training data (Assume label 0 as the majority class)
n_majority = 1000
n_minority = 1000

n_majority_1 = 1000
n_minority_1 = 1000

imbalanced_dfs = []
imbalanced_dfs_1 = []

label_0_class = df_train[df_train["label"] == 2] #We pick class 0 as the majority class
imbalanced_dfs.append(label_0_class.sample(n_majority, random_state=42))
imbalanced_dfs_1.append(label_0_class.sample(n_majority_1, random_state=42))

for label in sorted(df_train["label"].unique()):
    if label != 2:
        class_samples = df_train[df_train['label'] == label]
        imbalanced_dfs.append(class_samples.sample(n_minority, random_state=42))
        imbalanced_dfs_1.append(class_samples.sample(n_minority_1, random_state=42))

imbalanced_data = pd.concat(imbalanced_dfs)
# Shuffle the imbalanced dataset to mix the classes
imbalanced_data = imbalanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

imbalanced_data_5_1_ratio = pd.concat(imbalanced_dfs_1)
imbalanced_data_5_1_ratio = imbalanced_data_5_1_ratio.sample(frac=1).reset_index(drop=True)

print("\nBalanced Training Set Label Distribution:")
print(balanced_data['label'].value_counts().sort_index())

print("\nImbalanced Training Set Label Distribution:")
print(imbalanced_data['label'].value_counts().sort_index())




Balanced Training Set Label Distribution:
label
0    1000
1    1000
2    1000
3    1000
Name: count, dtype: int64

Imbalanced Training Set Label Distribution:
label
0    1000
1    1000
2    1000
3    1000
Name: count, dtype: int64


In [42]:
# Create a small testing set
n_rows_per_class = 1000
test_balanced_dfs = []
for label in sorted(df_test["label"].unique()):
    test_samples = df_test[df_test["label"] == label]
    test_balanced_dfs.append(test_samples.sample(n_rows_per_class, random_state=42))

testing_set = pd.concat(test_balanced_dfs)
testing_set = testing_set.sample(frac=1, random_state=42).reset_index(drop=True)



In [43]:
# Create a function to build the prompt strings on both balanced and imbalanced
label_map = {
    0: "world",
    1: "sports",
    2: "business",
    3: "sci/tech"
}

def build_shots_prompt(train_df, shots=None, imbalanced_ratio=None):
    prompt_lines = ["""You are a strict text classification system.

                        Task:
                        Classify the text into exactly one of these categories:
                        - world
                        - sports
                        - business
                        - sci/tech

                        Rules:
                        1. Output only one label.
                        2. The label must be exactly one of: world, sports, business, sci/tech.
                        3. The label must be lowercase.
                        4. Do not output explanations, punctuation, or extra text."""]
    
    if shots:
        # Build a balanced prompt
        for label in sorted(train_df['label'].unique()):
            class_samples = train_df[train_df['label'] == label].sample(shots, random_state=42)
            for _, row in class_samples.iterrows():
                prompt_lines.append(f"Text: {row['text']}")
                prompt_lines.append(f"Category: {label_map[row['label']]}")
                prompt_lines.append("")  # Add a blank line between examples
                
    elif imbalanced_ratio:
        # Build an imbalanced prompt based on the provided ratios
        for label, n_shots in imbalanced_ratio.items():
            class_samples = train_df[train_df['label'] == label].sample(n_shots, random_state=42)
            for _, row in class_samples.iterrows():
                prompt_lines.append(f"Text: {row['text']}")
                prompt_lines.append(f"Category: {label_map[row['label']]}")
                prompt_lines.append("")  # Add a blank line between examples
    else:
        raise ValueError("Must provide either 'shots_per_class' or 'imbalanced_ratios'")
        
    # Join all lines into a single string
    prompt_str = "\n".join(prompt_lines)
    return prompt_str


In [44]:
sorted(imbalanced_data["label"].unique())

[np.int64(0), np.int64(1), np.int64(2), np.int64(3)]

In [49]:
# imbalanced_data["label"] = imbalanced_data["label"].map(label_map)
# imbalanced_data["prompted_text"] = imbalanced_data.apply(
#     lambda row: build_shots_prompt(
#         imbalanced_data,
#         shots=2
#     ) + f"\nText: {row['text']}\nCategory:",
#     axis=1
# )
testing_set["prompted_text"] = testing_set.apply(
    lambda row: build_shots_prompt(
        testing_set,
        shots=2
    ) + f"\nText: {row['text']}\nCategory:",
    axis=1
)

In [53]:
# drop columns text
# imbalanced_data = imbalanced_data.drop(columns=["text"])
# imbalanced_data["label"] = imbalanced_data["label"].map(label_map)
# testing_set = testing_set.drop(columns=["text"])
testing_set["label"] = testing_set["label"].map(label_map)

In [54]:
# Save datasets to parquet for later use
balanced_data.to_parquet('../Data/ag_news/ag_news_train_balanced.parquet')
imbalanced_data.to_parquet('../Data/ag_news/ag_news_train_imbalanced_99_to_1.parquet')
testing_set.to_parquet('../Data/ag_news/ag_news_test_small.parquet')
imbalanced_data_5_1_ratio.to_parquet('../Data/ag_news/ag_news_train_imbalanced_49_to_1_ratio.parquet')

In [36]:
def classify(text, examples_prompt, model_name, api_key):
    full_prompt = f"{examples_prompt}\n\nNow classify this new text:\nText: {text}\nCategory:"
    payload = {
        "model": model_name, 
        "messages": [
            {"role": "user", "content": full_prompt}
        ],
        "temperature": 0.0,  
        "max_tokens": 1    
    }

    # The headers required by Open Router
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        response.raise_for_status() 
        
        result = response.json()
        # Extract the model's response, which is similar to the OpenAI API.
        prediction = result['choices'][0]['message']['content'].strip()
        pred_lines = prediction.splitlines()[0].lower()
        for label in label_map.values():
            if label.lower() in pred_lines:
                return label
        return prediction.strip()
        
    except requests.exceptions.RequestException as e:
        print(f"Request error for text '{text[:50]}...': {e}")
        time.sleep(5)  
        return "Error"
    except KeyError as e:
        print(f"Could not parse response for text '{text[:50]}...': {e}. Response: {result}")
        return "Error"


In [37]:
from sklearn.metrics import classification_report
import re

model = "nvidia/nemotron-nano-9b-v2:free"

# Define expected labels to map into the LLM's response
expected_labels = list(label_map.values())
expected_labels = [lab.lower() for lab in expected_labels]

balanced_prompt = build_shots_prompt(balanced_data, shots=2)
imbalanced_prompt = build_shots_prompt(balanced_data, imbalanced_ratio={0: 10, 1: 10, 2: 1, 3: 1})
imbalanced_prompt_5_to_1 = build_shots_prompt(balanced_data, imbalanced_ratio={0:5, 1:5, 2:1, 3:1})

print(f"--- EVALUATING {model} ---")
model_res = {}

def clean_prediction(pred, expected_labels):
    if not pred:
        return "Unknown"
    
    # Convert to lowercase and remove extra spaces
    pred = pred.lower().strip()
    
    
    pred = re.sub(r'[^\w\s]', '', pred)  
    pred = re.sub(r'\b(category|is|the|a|an)\b', '', pred)  
    pred = pred.strip()
    
    
    for label in expected_labels:
        if label.lower() in pred or pred in label.lower():
            return label
    
   
    variation_map = {
        'sport': 'Sports',
        'sci': 'Sci/Tech',
        'technology': 'Sci/Tech',
        'tech': 'Sci/Tech',
        'businesses': 'Business',
        'world news': 'World'
    }
    
    for variation, correct_label in variation_map.items():
        if variation in pred:
            return correct_label.lower()
    
    
    print(f"Unrecognized prediction: '{pred}' -> Mapping to 'Unknown'")
    return "Unknown"


accuracy_arr = []
for prompt_name, prompt in [("Balanced", balanced_prompt), ("Imbalanced with 10:1 Ratio", imbalanced_prompt), ("Imbalanced with 5:1 Ratio", imbalanced_prompt_5_to_1)]:
    print(f"Running with {prompt_name} prompt\n")
    
    curr_pred = 0
    n_rows = 20

    for _, row in testing_set.iloc[n_rows:41].iterrows():
        y_true = label_map[row["label"]].lower()
        raw_pred = classify(
            text=row["text"],
            examples_prompt=prompt,
            model_name=model,
            api_key=API_KEY
        )
        
       
        cleaned_pred = clean_prediction(raw_pred, expected_labels)
        
        if cleaned_pred == y_true:
            curr_pred += 1
        
        print(f"Sample: True='{y_true}', Raw='{raw_pred}', Cleaned='{cleaned_pred}'")
    
    acc = (curr_pred / n_rows) * 100
    accuracy_arr.append(acc)

    print(f"\n=== {prompt_name} Accuracy: {acc:.4f} ===\n")
    
    time.sleep(60)  

print("Evaluation done")


--- EVALUATING nvidia/nemotron-nano-9b-v2:free ---
Running with Balanced prompt

Sample: True='sports', Raw='sports', Cleaned='sports'
Sample: True='sports', Raw='sports', Cleaned='sports'
Sample: True='sci/tech', Raw='business', Cleaned='business'
Sample: True='sports', Raw='sports', Cleaned='sports'
Sample: True='business', Raw='business', Cleaned='business'
Sample: True='world', Raw='world', Cleaned='world'
Sample: True='sci/tech', Raw='business', Cleaned='business'
Sample: True='sci/tech', Raw='sci/Tech', Cleaned='sci/tech'
Sample: True='business', Raw='business', Cleaned='business'
Sample: True='sports', Raw='sports', Cleaned='sports'
Sample: True='business', Raw='sci/Tech', Cleaned='sci/tech'
Sample: True='sports', Raw='world', Cleaned='world'
Sample: True='sci/tech', Raw='sci/Tech', Cleaned='sci/tech'
Sample: True='business', Raw='business', Cleaned='business'
Sample: True='business', Raw='business', Cleaned='business'
Sample: True='world', Raw='world', Cleaned='world'
Sample: T